In [29]:
import pandas as pd
import openpyxl
import h5py
import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
from sklearn.model_selection import train_test_split
import sys

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [30]:
is_local = True # todo

# Experiment
seed = 1000 if is_local else int(sys.argv[-2])
torch.manual_seed(seed)
image_size = 360

# Data: which wavenumbers are even allowed to be considered?
wv_start = 0
wv_end = 340

# Data loading
batch_size= 64
patch_dim = 1
samples_to_train = 100000 # todo 10000

channels_used = np.s_[...,:] # used only when r_method = 'fixed'
print(channels_used)

(Ellipsis, slice(None, None, None))


In [31]:
def csf_fp(filepath):
    return filepath.replace('D:/datasets','D:/datasets' if is_local else './')

master = pd.read_excel(csf_fp(rf'D:/datasets/pcuk2023_qcl2025_whole_core_raw/master_sheet.xlsx'))
slide = master['slide'].to_numpy()
patient_id = master['patient_id'].to_numpy()
hdf5_filepaths = np.array([csf_fp(fp) for fp in master['hdf5_filepath']])
annotation_filepaths = np.array([csf_fp(fp) for fp in master['annotation_filepath']])
mask_filepaths = np.array([csf_fp(fp) for fp in master['mask_filepath']])
wavenumbers = np.load(csf_fp(f'D:/datasets/pcuk2023_ftir_whole_core/wavenumbers.npy'))[wv_start:wv_end]
wavenumbers_used = wavenumbers[channels_used]

annotation_class_colors = np.array([[0,255,0],[128,0,128],[255,0,255],[0,0,255],[255,165,0],[255,0,0]])
annotation_class_names = np.array(['epithelium_n','stroma_n','epithelium_c','stroma_c','corpora_amylacea','blood'])
n_classes = len(annotation_class_names)
print(f"Loaded {len(slide)} cores")
print(f"Using {len(wavenumbers_used)}/{len(wavenumbers)} wavenumbers")

Loaded 166 cores
Using 340/340 wavenumbers


In [36]:
class ftir_patching_dataset(torch.utils.data.Dataset):
    def __init__(self,hdf5_filepaths, patient_ids,mask_filepaths, annotation_filepaths, channels_use,
                 patch_dim=25, augment=True,):
        
        # Define data paths
        self.hdf5_filepaths = hdf5_filepaths
        self.patient_ids = patient_ids
        self.mask_filepaths = mask_filepaths
        self.annotation_filepaths = annotation_filepaths
        self.augment = augment
        
        # patch dimensions
        self.patch_dim = patch_dim
        self.patch_minus = patch_dim //2; self.patch_plus = 1 + (patch_dim // 2)
        self.channels = channels_use
        
        # class data
        self.annotation_class_colors = annotation_class_colors
        self.annotation_class_names = annotation_class_names
        self.total_sampled = torch.zeros(len(self.annotation_class_colors))
        
        # Open every core hdf5 file
        self.open()
        
    def __len__(self):
        return self.total_pixels
    
    def __getitem__(self,idx):
        # get patch data
        row = self.rows[idx]
        col = self.cols[idx]
        cidx = self.cidxs[idx]
        pidx = self.pidxs[idx]
        label = self.tissue_classes[idx]
        self.total_sampled[label] += 1
        
        # Are dimensions of patch okay
        idx_u = row - self.patch_minus
        idx_d = row + self.patch_plus
        idx_l = col - self.patch_minus
        idx_r = col + self.patch_plus
        pad_u = max(-idx_u,0); idx_u = max(idx_u,0)
        pad_d = max(idx_d-image_size,0); idx_d = min(idx_d,image_size)
        pad_l = max(-idx_l,0); idx_l = max(idx_l,0)
        pad_r = max(idx_r-image_size,0); idx_r = min(idx_r,image_size)
        
        # get patch
        patch = torch.from_numpy(
            self.hdf5_files[cidx]['spectra'][idx_u:idx_d,idx_l:idx_r,*self.channels],
        ).permute(2,0,1)
        patch *= torch.from_numpy(
            self.hdf5_files[cidx]['mask'][idx_u:idx_d,idx_l:idx_r,],
        ).unsqueeze(0)
        
        # pad patch
        patch = torch.nn.functional.pad(patch,(pad_l,pad_r,pad_u,pad_d,0,0))
        
        if self.augment:
            patch = self.transforms(patch)
        return patch,label,pidx

    # split annotations from H x W x 3 to C x H x W, one/zerohot along C dimension
    def split_annotations(self,annotations_img):
        split = torch.zeros((len(self.annotation_class_colors),*annotations_img.shape[:-1]))
        for c,col in enumerate(annotation_class_colors):
            split[c,:,:] = torch.from_numpy(np.all(annotations_img == self.annotation_class_colors[c],axis=-1)) 
        return split
    
    # open every file 
    def open(self):
        self.hdf5_files = []
        self.tissue_classes = []
        self.rows = []
        self.cols = []
        self.cidxs = []
        self.pidxs = []
        
        # for every core in dataset,
        for cidx in range(0,len(self.hdf5_filepaths)):
            # open annotations and remove edges and non-tissue px
            annotation = self.split_annotations(cv2.imread(self.annotation_filepaths[cidx])[:,:,::-1])
            mask = torch.from_numpy(cv2.imread(self.mask_filepaths[cidx])[:,:,1]) / 255
            annotation *= mask
            pidx = self.patient_ids[cidx]
            # for every class,
            for cls in range(len(annotation_class_names)):
                # get location of annotations, append to lists
                r,c = torch.where(annotation[cls])
                num_cls = annotation[cls].sum().int().item()
                self.tissue_classes.extend([cls,]*num_cls)
                self.cidxs.extend([cidx,]*num_cls)
                self.pidxs.extend([pidx,]*num_cls)
                self.rows.extend(r)
                self.cols.extend(c)
            # add open hdf5 file to list
            self.hdf5_files.append(h5py.File(self.hdf5_filepaths[cidx],'r'))
                
        # construct data tensors
        self.rows = torch.Tensor(self.rows).int()
        self.cols = torch.Tensor(self.cols).int()
        self.tissue_classes = torch.Tensor(self.tissue_classes).long()
        self.cidxs = torch.Tensor(self.cidxs).int()
        self.pidxs = torch.Tensor(self.pidxs).int()
        self.total_pixels = len(self.cidxs)

    # close every open hdf5 file
    def close(self):
        for cidx in range(len(self.hdf5_files)):
            self.hdf5_files[cidx].close()
        self.hdf5_files = []
        self.tissue_classes = []
        self.xs = []
        self.ys = []

In [37]:
dataset = ftir_patching_dataset(
    hdf5_filepaths, patient_id, mask_filepaths, annotation_filepaths, channels_used,
    patch_dim = patch_dim, augment=False,
)

# Instiantiate data loaders
_, class_counts = np.unique(dataset.tissue_classes, return_counts=True)
class_weights = 1 / class_counts
class_weights = class_weights[dataset.tissue_classes]
sampler = torch.utils.data.WeightedRandomSampler(class_weights, len(class_weights), replacement=True)

loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
loader_even = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=sampler,drop_last=True)
print(f"loader sizes:\n\tnormal unbalanced: {len(loader)}\n\teven: {len(loader_even)}")

loader sizes:
	normal unbalanced: 16816
	even: 16816


In [38]:
data_odd = []
labels_odd = []
pids_odd = []
for iter in range(0, (samples_to_train // batch_size) + 1):
    for bidx, (data, label, pid) in enumerate(loader):
        print(f"{bidx}/{(samples_to_train // batch_size)}",end="\r")
        data_odd.append(data.squeeze().cpu().numpy())
        labels_odd.append(label.squeeze().cpu().numpy())
        pids_odd.append(pid.squeeze().cpu().numpy())
        
        if iter*len(loader) + bidx*batch_size > samples_to_train:
            end = True
            break
    if end:
        break
data_odd = np.concatenate(data_odd,axis=0)
labels_odd = np.concatenate(labels_odd,axis=0)
pids_odd = np.concatenate(pids_odd,axis=0)
print(f"Assembled data with {len(data_odd)} samples and {len(labels_odd)} labels and {len(pids_odd)} pids.")

Assembled data with 100096 samples and 100096 labels and 100096 pids.


In [40]:
data_even = []
labels_even = []
pids_even = []
for iter in range(0, (samples_to_train // batch_size) + 1):
    for bidx, (data, label, pid) in enumerate(loader_even):
        print(f"{bidx}/{(samples_to_train // batch_size)}",end="\r")
        data_even.append(data.squeeze().cpu().numpy())
        labels_even.append(label.squeeze().cpu().numpy())
        pids_even.append(pid.squeeze().cpu().numpy())
        
        if iter*len(loader_even) + bidx*batch_size > samples_to_train:
            end = True
            break
    if end:
        break
data_even = np.concatenate(data_even,axis=0)
labels_even = np.concatenate(labels_even,axis=0)
pids_even = np.concatenate(pids_even,axis=0)
print(f"Assembled data with {len(data_even)} samples and {len(labels_even)} labels and {len(pids_even)} pids.")

Assembled data with 100096 samples and 100096 labels and 100096 pids.


In [41]:
np.save("./data/data_even.npy", data_even)
np.save("./data/labels_even.npy", labels_even)
np.save("./data/pids_even.npy", pids_even)
np.save("./data/data_odd.npy", data_odd)
np.save("./data/labels_odd.npy", labels_odd)
np.save("./data/pids_odd.npy", pids_odd)